In [503]:
import pandas as pd
import krippendorff
import numpy as np
import re

In [ ]:
#http://ict.usc.edu/pubs/Inter-Coder%20Agreement%20for%20Computational%20Linguistics.pdf
#https://towardsdatascience.com/inter-rater-agreement-kappas-69cd8b91ff75

### Calculating reliability on final training set (N = 20)

In [662]:


def get_percent_agreement(x):
    if x[df.columns[0]] == x[df.columns[1]] == x[df.columns[2]]:
         return(100)
    elif x[df.columns[0]] == x[df.columns[1]] != x[df.columns[2]]:
        return(66.66)
    elif x[df.columns[0]] != x[df.columns[1]] == x[df.columns[2]]:
        return(66.66)
    elif x[df.columns[0]] == x[df.columns[2]] != x[df.columns[1]]:
        return(66.66)
    else: 
        return(0)

def get_percent_agreement_frames(df):
    df['1s'] = df.isin([1.0]).sum(axis=1)
    df['1s'] = df['1s'].replace(0, 3)
    df['agreement'] = (df['1s'] / 3 ) * 100
    return df['agreement'].mean()

def recode_maintopics(df):
    ''' returns recoded main topics '''

    df['C1: V2'] = df['C1: V2'].str[:-4]
    df['C2: V2'] = df['C2: V2'].str[:-4]
    df['C3: V2'] = df['C3: V2'].str[:-4]
    return df

In [354]:
def recode_maintopics(df):
    ''' returns recoded main topics '''

    df['C1: V2'] = df['C1: V2'].str[:-4]
    df['C2: V2'] = df['C2: V2'].str[:-4]
    df['C3: V2'] = df['C3: V2'].str[:-4]
    return df

#### get Kalpa and % agreement for frames

In [593]:
# Frames
df = pd.read_excel("data/raw/RPA_frames_ICR.xlsx", skiprows=1)
df.columns

#attribution = df[['C1: V4.1', 'C2: V4.1', 'C3: V4.1']][:20].transpose()
print("Attribution of responsibiity:")
attribution = df[['C1: V4.1', 'C2: V4.1', 'C3: V4.1']][:20].transpose()
print("Krippendorff:")
print(krippendorff.alpha(attribution))
print("Percentage agreement:")
print(get_percent_agreement_frames(attribution.transpose()) )

print()

print("Human Interest:")
humaninterest = df[['C1: V5.1', 'C2: V5.1', 'C3: V5.1']][:20].transpose()
print("Krippendorff:")
print(krippendorff.alpha(humaninterest))
print("Percentage agreement:")
print(get_percent_agreement_frames(humaninterest.transpose()) )

print()

conflictframe = df[['C1: V6.1', 'C2: V6.1', 'C3: V6.1']][:20].transpose()
print("Conflict Frame:")
print(krippendorff.alpha(conflictframe))
print("Percentage agreement:")
print(get_percent_agreement_frames(conflictframe.transpose()) )

print()


economics = df[['C1: V8.1', 'C2: V8.1', 'C3: V8.1']][:20].transpose()
print("Economic consequences frame")
print(krippendorff.alpha(economics ))
print("Percentage agreement:")
print(get_percent_agreement_frames(economics.transpose()) )


Attribution of responsibiity:
Krippendorff:
0.3572984749455338
Percentage agreement:
84.99999999999999

Human Interest:
Krippendorff:
0.5809659090909092
Percentage agreement:
86.66666666666666

Conflict Frame:
0.5745192307692308
Percentage agreement:
93.33333333333333

Economic consequences frame
0.7882775119617225
Percentage agreement:
91.66666666666666


#### get Kalpa and % agreement for topics

In [594]:
df = pd.read_excel("data/raw/RPA_topics_ICR.xlsx", skiprows=1)
df = df[['C1: V2', 'C2: V2', 'C3: V2']][:19].astype(str)

df['C4: V2'] = np.nan
df['C5: V2'] = np.nan

df = recode_maintopics(df)
df = df[['C1: V2', 'C2: V2', 'C3: V2']].astype(int)
df = df.transpose()

print("Krippendorff alpha for topics is:")
print(krippendorff.alpha(df))

# Calculate percentage agreement
df = df.transpose()
               
len(df)


Krippendorff alpha for topics is:
0.6227780905654308


19

#### Getting reliability based on duplicate dataset

In [584]:
df2 = pd.read_csv("data/raw/RPA_coding.csv", skiprows = 1)

df2['n'] = 1
df2.rename(columns={'Document nummer (voor kranten staat in bestand voor kamervragen bijvoorbeeld ah-tk-20012002-89 (z...':'docnr'}, 
                 inplace=True)
df2.columns
d = df2[df2.duplicated('docnr',keep=False)]

d = d[d.Codeursnaam.notnull()]
d = d[d.docnr.notnull()]
d[['Codeursnaam', 'docnr']]

icr = d
icr = icr[~icr.duplicated(['docnr', 'Codeursnaam'])]

icr.sort_values('docnr')[['Codeursnaam', 'docnr']]

icr = icr[~icr.duplicated(['docnr', 'Codeursnaam'])]
icr = icr[icr.duplicated('docnr',keep=False)]

#icr[['Codeursnaam', 'docnr', icr.columns[13]]].sort_values('docnr')

#icr = icr[icr['Krantenbericht of kamervraag?'] == 1]
#icr = icr[icr.duplicated(['docnr', icr.columns[13]],keep=False)]
#icr = icr[icr.duplicated(['docnr', 'Datum? (DD-MM-JJJJ)'],keep=False)]
#icr[icr.duplicated(['docnr', icr.columns[13]],keep=False)][['Codeursnaam', 'docnr', icr.columns[13], 'Datum? (DD-MM-JJJJ)']].sort_values('docnr')

conflict = icr.columns[26]
human_interest = icr.columns[23]
att_res = icr.columns[20]
economic = icr.columns[29]

icr.rename(columns={human_interest : 'human_interest', att_res : 'attribution_resp', conflict : 'conflict', economic : 'economic_consequences', 'Datum? (DD-MM-JJJJ)' : 'publication_date'}, inplace=True)

# get dates

error = icr.publication_date[pd.isnull(pd.to_datetime(icr.publication_date, errors ='coerce'))]
indexes_to_keep = set(range(icr.shape[0])) - set(error.index)
icr = icr.take(list(indexes_to_keep))
icr['publication_date'] = pd.to_datetime(icr.publication_date, errors ='coerce')
icr['year'] = pd.DatetimeIndex(icr['publication_date']).year
icr['month'] = pd.DatetimeIndex(icr['publication_date']).month
icr['day'] = pd.DatetimeIndex(icr['publication_date']).day

icr = icr[icr.duplicated(['docnr','year','month','day', icr.columns[13],icr.columns[12], icr.columns[15]],keep=False)]

#icr = icr[icr.duplicated(['docnr', 'day'],keep=False)]
#icr[icr.duplicated(['docnr', icr.columns[13]],keep=False)][['Codeursnaam', 'docnr', 'year', 'month',icr.columns[13], 'year']].sort_values('docnr')

# make sure doc nr and newspaper are correct
#icr = icr[icr.duplicated(['docnr',  icr.columns[13]],keep=False)]
icr[icr.columns[12]].value_counts()
icr[['docnr', icr.columns[12]]]
fullsample = icr

icr = icr[icr[icr.columns[12]] == 1]
print("The N of the sample w only dubble coded news articles (used to calculated ICR for frames) is: {}".format(len(icr)))


The N of the sample w only dubble coded news articles (used to calculated ICR for frames) is: 22


In [585]:
#for number, frame in zip(['V5.1', 'V4.1', 'V6.1','V8.1'], ['human_interest', 'attribution_resp', 'conflict', 'economic_consequences']):
#    print(frame, number)
#    df = icr.pivot(index='docnr', columns='Codeursnaam', values=frame)  
#    df.rename(columns={ 1.0: 'C1: '+ number, 3.0: 'C2: '  + number, 4.0: 'C3: ' + number, 5.0: 'C4: ' + number, 6.0: 'C5: '  + number}, inplace=True )
#    df

topics = fullsample.pivot(index='docnr', columns='Codeursnaam', values=icr.columns[17])
topics.rename(columns={  3.0: 'C3: V2', 4.0: 'C4: V2', 5.0: 'C5: V2', 6.0: 'C6: V2'}, inplace=True )

topics.reset_index(inplace=True)

topics['C1: V2'] = np.nan
topics['C2: V2'] = np.nan
topics['C3: V2'] = topics['C3: V2'].str[:-2]
topics['C4: V2'] = topics['C4: V2'].str[:-2]
topics['C5: V2'] = topics['C5: V2'].str[:-2]
topics['C6: V2'] = topics['C6: V2'].str[:-2]

topics = topics.replace(r'^\s*$', np.nan, regex=True)
        
topics['C3: V2'] = pd.to_numeric(topics['C3: V2'], errors='coerce')
topics['C1: V2'] = pd.to_numeric(topics['C1: V2'], errors='coerce')
topics['C2: V2'] = pd.to_numeric(topics['C2: V2'], errors='coerce')
topics['C4: V2'] = pd.to_numeric(topics['C4: V2'], errors='coerce')
topics['C5: V2'] = pd.to_numeric(topics['C5: V2'], errors='coerce')
topics['C6: V2'] = pd.to_numeric(topics['C6: V2'], errors='coerce')

df['C4: V2'] = np.nan
df['C5: V2'] = np.nan
df['C6: V2'] = np.nan

df_ = pd.concat([df, topics])

print("The N of the sample with both dubble coded Parliamentary questions and news articles (used to calculated ICR for topics) is: {}".format(len(df_)))
print()


df_ = df_[['C1: V2', 'C2: V2', 'C3: V2', 'C4: V2', 'C5: V2', 'C6: V2']]

print(krippendorff.alpha(df_.transpose()))

The N of the sample with both dubble coded Parliamentary questions and news articles (used to calculated ICR for topics) is: 395

0.7505034807878472


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [586]:
#icr.pivot(index='docnr', columns='Codeursnaam', values='conflict') 
#ar
#humaninterest
#ar.columns
#ar[1.0]
#ar.rename(columns={ 1.0: 'C1: V4.1' , 3.0: 'C2: V4.1', 4.0: 'C3: V4.1', 5.0: 'C4: V4.1', 6.0: 'C5: V4.1'}, inplace=True )
#conflictframe

In [680]:
#'attribution_resp'
#human_interest
#conflict
#economic_consequences

def get_df_per_frame(frame):
    ar = icr.pivot(index='docnr', columns='Codeursnaam', values=frame) 
    if frame == 'attribution_resp':
        ar.rename(columns={ 1.0: 'C1: V4.1' , 3.0: 'C2: V4.1', 4.0: 'C3: V4.1', 5.0: 'C4: V4.1', 6.0: 'C5: V4.1'}, inplace=True )
    if frame == 'human_interest':
        ar.rename(columns={ 1.0: 'C1: V5.1' , 3.0: 'C2: V5.1', 4.0: 'C3: V5.1', 5.0: 'C4: V5.1', 6.0: 'C5: V5.1'}, inplace=True )
    if frame == 'conflict':
        ar.rename(columns={ 1.0: 'C1: V6.1' , 3.0: 'C2: V6.1', 4.0: 'C3: V6.1', 5.0: 'C4: V6.1', 6.0: 'C5: V6.1'}, inplace=True )
    if frame == 'economic_consequences':
        ar.rename(columns={ 1.0: 'C1: V8.1' , 3.0: 'C2: V8.1', 4.0: 'C3: V8.1', 5.0: 'C4: V8.1', 6.0: 'C5: V8.1'}, inplace=True )
    return ar

ar = get_df_per_frame('attribution_resp')

attribution = attribution.transpose() 
attribution['C4: V4.1'] = np.nan
attribution['C5: V4.1'] = np.nan

ar_ = pd.concat([ar, attribution])
print(krippendorff.alpha(ar_.transpose()))

print(len(ar_))

0.34340456890198967
31


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [588]:
hi= get_df_per_frame('human_interest')
humaninterest = humaninterest.transpose() 
humaninterest['C4: V5.1'] = np.nan
humaninterest['C5: V5.1'] = np.nan

hi_ = pd.concat([hi, humaninterest])
print(krippendorff.alpha(hi_.transpose()))
print(len(hi_))

0.49414519906323184
31


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [663]:
df['num_uniq'] = [len(set(v[pd.notna(v)].tolist())) for v in df.values]

In [661]:
hi__ =hi_.apply(pd.value_counts, axis=1).fillna(0)
hi__['ww'] =  ( hi__[2.0] * hi__[1.0] ) + 100
        
def get_percent_agreement(row):
    if row['ww'] == 100.0:
        return(100)
    elif row['ww'] == 101.0:
        return(0)
    elif row['ww'] == 102.0:
        return(66.6)
    else: 
        return(0)

print("percent agreement human interest: {}".format( hi__.apply(lambda row: get_percent_agreement(row),axis=1).mean() ))

percent agreement human interest: 84.93548387096774


In [589]:
cf= get_df_per_frame('conflict')
conflictframe= conflictframe.transpose() 
conflictframe['C4: V6.1'] = np.nan
conflictframe['C5: V6.1'] = np.nan

cf_ = pd.concat([cf, conflictframe])
print(krippendorff.alpha(cf_.transpose()))

print(len(cf_))

print("length of orginal dataset: {}, lenght of the dublicates: {}".format(len(conflictframe),len(cf)))

0.5224760501105379
31
length of orginal dataset: 20, lenght of the dublicates: 11


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [575]:
ec= get_df_per_frame('economic_consequences')
economics= economics.transpose() 
economics['C4: V8.1'] = np.nan
economics['C5: V8.1'] = np.nan

ec_ = pd.concat([ec, economics])
print(krippendorff.alpha(ec_.transpose()))

0.6837996096291477


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [211]:
def label_topic(row):
    if row['main_topic'] == '1':
        return 'Macro-economie en belastingen'
    if row['main_topic'] == '2':
        return 'Burgerrechten en vrijheden'
    if row['main_topic'] == '3':
        return 'Gezondheid'
    if row['main_topic'] == '4':
        return 'Landbouw en Visserij'
    if row['main_topic'] == '5':
        return 'Arbeid'
    if row['main_topic'] == '6':
        return 'Onderwijs'
    if row['main_topic'] == '7':
        return 'Milieu'
    if row['main_topic'] == '8':
        return 'Energiebeleid'
    if row['main_topic'] == '9':
        return 'Immigratie en integratie'
    if row['main_topic'] == '10':
        return 'Verkeer en vervoer'
    if row['main_topic'] == '11':
        return 'Unkown'
    if row['main_topic'] == '12':
        return 'Justitie, Rechtspraak, Criminaliteit'
    if row['main_topic'] == '13':
        return 'sociale Zaken'
    if row['main_topic'] == '14':
        return 'Gemeenschapsontwikkeling, huisvestingsbeleid en stedelijke planning'
    if row['main_topic'] == '15':
        return 'Ondernemingen, Bankwezen en binnenlandse handel '
    if row['main_topic'] == '16':
        return 'Defensie'
    if row['main_topic'] == '17':
        return 'Wetenschappelijk onderzoek, technologie en communicatie'
    if row['main_topic'] == '18':
        return 'Buitenlandse handel'
    if row['main_topic'] == '19':
        return 'Buitenlandse zaken en ontwikkelingssamenwerking'
    if row['main_topic'] == '20':
        return 'Functioneren democratie en openbaar bestuur'
    if row['main_topic'] == '21':
        return 'Ruimtelijke ordening, publiek natuur- en waterbeheer'
    if row['main_topic'] == '22':
        return 'Unkown 2'
    if row['main_topic'] == '23':
        return 'Kunst, cultuur en entertainment'
    if row['main_topic'] == '00':
        return 'Toegevoegde codes voor media'


def recode_maintopics(df):
    ''' returns recoded main topics '''

    df['main_topic'] = df['topic_number'].str[:-2]
    df['main_topic_label'] = df.apply (lambda row: label_topic (row),axis=1)
    df[['main_topic','main_topic_label']][0:5]
    return df


icr['topic_number'] = icr[icr.columns[17]]
df = recode_maintopics(icr)

df['main_topic'] = pd.to_numeric(df['main_topic'], errors='coerce')
r = df.pivot(index='docnr', columns='Codeursnaam', values='main_topic')
#print("length of the df: {}".format(len(list(r_.columns)) ))
r = r.values

krippendorff.alpha(r)
#import numpy as np
#df['main_topic'] = df['main_topic'].replace('', np.nan)
#df['main_topic'].astype(int)

0.0024608866213596814